In [1]:
import numpy as np
from discretize import TensorMesh
import matplotlib.pyplot as plt
from pymatsolver import PardisoSolver
import SimPEG.EM.TDEM as tdem
from SimPEG import (Utils,Mesh, Maps, DataMisfit, Regularization,
                    Optimization, Inversion, InvProblem, Directives)
from SimPEG import EM, Mesh, Utils

In [7]:
datadir = "./TEM_data/"

dobs = np.load(datadir + 'dobs.npy')
time = np.load(datadir + 'times.npy')
locs = np.load(datadir + 'locs.npy')
m0 = np.load(datadir + 'sigma0.npy')
actind =np.load(datadir + 'actind_2d.npy')
mesh_3d = TensorMesh.readUBC(datadir + 'mesh_3d.msh')
mesh_2d = TensorMesh([mesh_3d.hx,mesh_3d.hz],x0=[mesh_3d.x0[0],mesh_3d.x0[2]])


In [3]:
map_2Dto3D = Maps.Surject2Dto3D(mesh_3d)
expmap = Maps.ExpMap(mesh_2d)
actmap = Maps.InjectActiveCells(mesh_2d, indActive=actind, valInactive=np.log(3))
actmap_plot = Maps.InjectActiveCells(mesh_2d, indActive=actind, valInactive=np.nan)
mapping = map_2Dto3D * expmap * actmap
#mapping_plot = map_2Dto3D * expmap * actmap_plot

In [4]:
srcList = []
for location in locs:
    rx_z = tdem.Rx.Point_dbdt(location, time, 'z')
    src = tdem.Src.CircularLoop([rx_z], orientation='z', loc=location)
    srcList.append(src)
timesteps = [(1e-4, 5),(5e-4, 5),(1e-3,10),(2e-3,5)]
#timesteps = time
survey = tdem.Survey(srcList)

In [5]:
prb = EM.TDEM.Problem3D_e(mesh_3d, Solver=PardisoSolver, verbose=False, timeSteps=timesteps, sigmaMap=mapping)
survey = EM.TDEM.Survey(srcList)
survey.pair(prb)

In [8]:
survey.std = 0.05
survey.eps = 1e-12
survey.dobs = dobs

In [9]:
%%time
dmisfit = DataMisfit.l2_DataMisfit(survey)

model_map = Maps.IdentityMap(nP = int(actind.sum())) 
reg = Regularization.Tikhonov(mesh_2d,mapping = model_map,indActive=actind ,
                              alpha_s = 1/mesh_2d.hx.min()**2, alpha_x = 10., alpha_y = 10.)

opt = Optimization.ProjectedGNCG(
    maxIter=25, lower=-4.0, upper=5.0,maxIterLS=20, maxIterCG=10, tolCG=1e-3
)

invProb = InvProblem.BaseInvProblem(dmisfit, reg, opt)
beta = Directives.BetaSchedule(coolingFactor=5, coolingRate=1)
betaest = Directives.BetaEstimate_ByEig(beta0_ratio=0.001)
target = Directives.TargetMisfit()

save_model = Directives.SaveModelEveryIteration()
save = Directives.SaveOutputEveryIteration()
directiveList = [beta,betaest, target, save_model, save]

inv = Inversion.BaseInversion(invProb, directiveList = directiveList)

sig0 = 3
mref = np.log(sig0)*np.ones(actmap.nP)
m0 = mref.copy()

mopt = inv.run(m0)

SimPEG.InvProblem will set Regularization.mref to m0.

    SimPEG.InvProblem is setting bfgsH0 to the inverse of the eval2Deriv.
    ***Done using same Solver and solverOpts as the problem***
SimPEG.SaveModelEveryIteration will save your models as: '.\###-InversionModel-2022-09-17-17-14.npy'
SimPEG.SaveOutputEveryIteration will save your inversion progress as: '###-InversionModel-2022-09-17-17-14.txt'
model has any nan: 0
=============================== Projected GNCG ===============================
  #     beta     phi_d     phi_m       f      |proj(x-g)-x|  LS    Comment   
-----------------------------------------------------------------------------
x0 has any nan: 0
   0  3.90e-05  7.63e+03  0.00e+00  7.63e+03    6.69e+01      0              
   1  7.79e-06  3.79e+03  4.44e+03  3.79e+03    6.82e+01      0              
   2  1.56e-06  2.73e+03  5.56e+03  2.73e+03    6.82e+01      0              
   3  3.12e-07  2.49e+03  7.09e+03  2.49e+03    5.68e+01      2   Skip BFGS  
   4  6.2

In [12]:
np.save('TEMinversion_result' +'/mopt', mopt)
np.save('TEMinversion_result' +'/dpred', invProb.dpred)

In [ ]:
# print diffusion distance and make sure mesh padding goes beyond that
print("diffusion distance: {:1.2e} m".format(1260*np.sqrt(3*2e-3)))
print("print mesh extent : {:1.2e} m".format(mesh_2d.hy[-10:].sum()))